In [1]:
import os
from utils import *
import optuna

In [2]:
data, usermap, itemmap, users = load_data2()
data_train, data_val=split_data2(data, 0.2)

In [10]:
rat_rank_study = optuna.create_study(
    study_name="P3Beta+Easer",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-09 09:26:28,436] A new study created in RDB with name: P3Beta+Easer


In [4]:
rp3beta_study = optuna.create_study(
    study_name="P3Beta",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-09 09:21:05,475] Using an existing study with name 'P3Beta' instead of creating a new one.


In [5]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

rp3beta_recommender = RP3betaRecommender(data_train, verbose=False)
rp3beta_recommender.fit(**rp3beta_study.best_params)

In [6]:
easer_study = optuna.create_study(
    study_name="Easer",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-09 09:21:28,124] Using an existing study with name 'Easer' instead of creating a new one.


In [7]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

easer_recommender = EASE_R_Recommender(data_train)
easer_recommender.fit(**easer_study.best_params)

EASE_R_Recommender: URM Detected 213 ( 1.7%) users with no interactions.
EASE_R_Recommender: URM Detected 120 ( 0.5%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 3.44 min


In [12]:
from Recommenders.BaseRecommender import BaseRecommender
import scipy.sparse as sps
import numpy.linalg as LA

class DifferentLossScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1/norm*alpha + R2/norm*(1-alpha) where R1 and R2 come from
    algorithms trained on different loss functions.

    """

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        
        
        
    def fit(self, norm, alpha = 0.5):

        self.alpha = alpha
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        
        
        if norm_item_weights_1 == 0:
            norm_item_weights_1 = 0.0000001
            #raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            norm_item_weights_2 = 0.0000001
            #raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * (1-self.alpha)

        return item_weights



In [13]:
def objective(trial):
    norm = trial.suggest_categorical("norm", [1, 2, np.inf, -np.inf, None])
    alpha = trial.suggest_uniform("alpha", 0, 1)

    recommender_object = DifferentLossScoresHybridRecommender(data_train, rp3beta_recommender, easer_recommender)
    recommender_object.fit(
        norm = norm, 
        alpha = alpha
    )

    _, _, ev_map, _, _ = evaluator(recommender_object, data_train, data_val)
    
    return ev_map

rat_rank_study.optimize(objective, n_trials=100)

/tmp/ipykernel_328043/1942927178.py:3: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform("alpha", 0, 1)


DifferentLossScoresHybridRecommender: URM Detected 213 ( 1.7%) users with no interactions.
DifferentLossScoresHybridRecommender: URM Detected 120 ( 0.5%) items with no interactions.


[I 2024-01-09 09:27:45,081] Trial 1 finished with value: 0.08115946630574146 and parameters: {'norm': None, 'alpha': 0.03745962530001046}. Best is trial 1 with value: 0.08115946630574146.


DifferentLossScoresHybridRecommender: URM Detected 213 ( 1.7%) users with no interactions.
DifferentLossScoresHybridRecommender: URM Detected 120 ( 0.5%) items with no interactions.


[I 2024-01-09 09:28:10,020] Trial 2 finished with value: 0.08345394682270492 and parameters: {'norm': 2, 'alpha': 0.2622433110540575}. Best is trial 2 with value: 0.08345394682270492.


...

DifferentLossScoresHybridRecommender: URM Detected 213 ( 1.7%) users with no interactions.
DifferentLossScoresHybridRecommender: URM Detected 120 ( 0.5%) items with no interactions.


[I 2024-01-09 09:30:20,088] Trial 7 finished with value: 0.08407603306358193 and parameters: {'norm': inf, 'alpha': 0.9809686949772557}. Best is trial 4 with value: 0.08481124596740923.


DifferentLossScoresHybridRecommender: URM Detected 213 ( 1.7%) users with no interactions.
DifferentLossScoresHybridRecommender: URM Detected 120 ( 0.5%) items with no interactions.


[I 2024-01-09 09:30:38,148] Trial 8 finished with value: 0.08380965340958546 and parameters: {'norm': None, 'alpha': 0.3350812116487889}. Best is trial 4 with value: 0.08481124596740923.


DifferentLossScoresHybridRecommender: URM Detected 213 ( 1.7%) users with no interactions.
DifferentLossScoresHybridRecommender: URM Detected 120 ( 0.5%) items with no interactions.
